In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.email import EmailOperator
from airflow.exceptions import AirflowFailException
from datetime import datetime, timedelta
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine

Definição de argumentos

In [ ]:
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
}

Criação da DAG

In [ ]:
dag = DAG(
    'antaq_etl',
    default_args=default_args,
    description='ETL da base ANTAQ usando Airflow',
    schedule_interval='@daily',
    catchup=False,
)

Download dos arquivos

In [ ]:
def baixar_arquivos():
    BASE_URL = "https://web3.antaq.gov.br/ea/sense/download.html#pt"
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, "html.parser")
    
    links = soup.find_all("a", string="Clique aqui.")
    os.makedirs("/opt/airflow/dados_brutos", exist_ok=True)
    
    for link in links:
        href = link.get("href")
        if href and href.endswith(".zip"):
            nome_arquivo = link.find_previous("td").text.strip()
            ano = href.split("/")[-2]
            
            file_path = f"/opt/airflow/dados_brutos/{nome_arquivo}_{ano}.zip"
            with open(file_path, "wb") as f:
                f.write(requests.get(href).content)

Transformação dos dados

In [ ]:
def carregar_dados():
    engine = create_engine("sqlite:////opt/airflow/dags/db/antaq.db")
    df = pd.DataFrame({"coluna": [1, 2, 3]})
    df.to_sql("tabela_antaq", engine, if_exists="replace", index=False)


Tarefas do Airflow

In [ ]:
task_baixar = PythonOperator(
    task_id='baixar_arquivos',
    python_callable=baixar_arquivos,
    dag=dag,
)

task_transformar = PythonOperator(
    task_id='transformar_dados',
    python_callable=transformar_dados,
    dag=dag,
)

task_carregar = PythonOperator(
    task_id='carregar_dados',
    python_callable=carregar_dados,
    dag=dag,
)

Notificação via e-mail

In [ ]:
task_email = EmailOperator(
    task_id='enviar_email',
    to='cleber.aksenen@gmail.com',
    subject='ETL ANTAQ Finalizado',
    html_content='ETL concluído com sucesso.',
    dag=dag,
)

Definição da ordem das tarefas

In [ ]:
task_baixar >> task_transformar >> task_carregar >> task_email